In [10]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/home/david.yang1/.cache/huggingface/'
os.environ['HF_HOME'] = '/home/david.yang1/.cache/huggingface/'

In [11]:
# Load model directly
from transformers import BartTokenizer, BartForConditionalGeneration
import pandas as pd

In [12]:
# Load data
data = pd.read_csv("/home/david.yang1/autolit/viriation/data/pipeline_data/paper_flagging_data/bert_dataset.csv")
flagged_papers = data[data["label"] == 1]

# Initial testing

In [17]:
# Load BioBART model and tokenizer
model_name = 'GanjinZero/biobart-v2-base'  # Assuming BioBART is available with this name
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/892k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/666M [00:00<?, ?B/s]

In [30]:
from transformers import pipeline

# summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
summarizer = pipeline("summarization", model="GanjinZero/biobart-v2-base")

# text = "Omicron variant contains more than 32 amino acid mutations in the spike protein, including multiple vital amino acid mutations (K417N, T478K, E484A, N501Y, and D614G) that have been already detected in other VOCs of SARS-CoV-2 and proved to be associated with enhanced transmissibility, virulence, and greater resistance to the immune protection induced by COVID-19 vaccines."
# text = flagged_papers["text"].iloc[0]
text = "Rapid detection and tracking of Omicron variant of SARS-CoV-2 using CRISPR-Cas12a-based assayBackgroundThe newly emerged SARS-CoV-2 variant of concern (VOC) Omicron is spreading quickly worldwide, which manifests an urgent need of simple and rapid assay to detect and diagnose Omicron infection and track its spread.MethodsTo design allele-specific CRISPR RNAs (crRNAs) targeting the signature mutations in the spike protein of Omicron variant, and to develop a CRISPR-Cas12a-based assay to specifically detect Omicron variant.ResultsOur system showed a low limit of detection of 2 copies per reaction for the plasmid DNA of Omicron variant, and could readily detect Omicron variant in 5 laboratory-confirmed clinical samples and distinguish them from 57 SARS-CoV-2 positive clinical samples (4 virus isolates and 53 oropharyngeal swab specimens) infected with wild-type (N = 8) and the variants of Alpha (N = 17), Beta (N = 17) and Delta (N = 15).The testing results could be measured by fluorescent detector or judged by naked eyes.In addition, no cross-reaction was observed when detecting 16 clinical samples infected with 9 common respiratory pathogens.ConclusionsThe rapid assay could be easily set up in laboratories already conducting SARS-CoV-2 nucleic acid amplification tests and implemented routinely in resource-limited settings to monitor and track the spread of Omicron variant.IntroductionContinuing spread and evolution of SARS-CoV-2 have resulted in the emergence of various variants that have infected and killed millions of people.The newly emerged fifth variant of concern (VOC) Omicron was firstly reported in South Africa on November 24, 2021 and has been detected in many countries.Omicron variant contains more than 32 amino acid mutations in the spike protein, including multiple vital amino acid mutations (K417N, T478K, E484A, N501Y, and D614G) that have been already detected in other VOCs of SARS-CoV-2 and proved to be associated with enhanced transmissibility, virulence, and greater resistance to the immune protection induced by COVID-19 vaccines.The new features of Omicron manifested the importance of tracking its spread.Reverse transcription polymerase chain reaction (RT-PCR) has been widely used for diagnosing SARS-CoV-2 infection and genotyping SARS-CoV-2 variants."
print(text)

print(summarizer(text, max_length=100, min_length=30, do_sample=False))

Rapid detection and tracking of Omicron variant of SARS-CoV-2 using CRISPR-Cas12a-based assayBackgroundThe newly emerged SARS-CoV-2 variant of concern (VOC) Omicron is spreading quickly worldwide, which manifests an urgent need of simple and rapid assay to detect and diagnose Omicron infection and track its spread.MethodsTo design allele-specific CRISPR RNAs (crRNAs) targeting the signature mutations in the spike protein of Omicron variant, and to develop a CRISPR-Cas12a-based assay to specifically detect Omicron variant.ResultsOur system showed a low limit of detection of 2 copies per reaction for the plasmid DNA of Omicron variant, and could readily detect Omicron variant in 5 laboratory-confirmed clinical samples and distinguish them from 57 SARS-CoV-2 positive clinical samples (4 virus isolates and 53 oropharyngeal swab specimens) infected with wild-type (N = 8) and the variants of Alpha (N = 17), Beta (N = 17) and Delta (N = 15).The testing results could be measured by fluorescent

In [6]:
# Input text (biomedical paper abstract)
text = flagged_papers["text"].iloc[0]
# text = "Omicron variant contains more than 32 amino acid mutations in the spike protein, including multiple vital amino acid mutations (K417N, T478K, E484A, N501Y, and D614G) that have been already detected in other VOCs of SARS-CoV-2 and proved to be associated with enhanced transmissibility, virulence, and greater resistance to the immune protection induced by COVID-19 vaccines."

# Tokenize input
inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)

# print(tokenizer.decode(inputs[0], skip_special_tokens=True))

# Generate summary
summary_ids = model.generate(inputs, max_length=1024, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
print(summary_ids)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print(summary)

tensor([[    2, 18581,  3916,  2072,    35,   384, 15796,  2839, 17390,  6308,
            55,    87,  2107, 37788, 10395, 28513,    11,     5, 12490,  8276,
             6,   217,  1533,  4874, 37788, 10395, 31425,    36,   530, 36711,
           487,     6,   255, 37664,   530,     6,   381, 37810,   250,     6,
           234, 36144,   975,     6,     8,   211, 40082,   534,    43,    14,
            33,    57,   416, 12333,    11,    97,   468,  4571,    29,     9,
           208, 22210,    12,  8739,   846,    12,   176,     8,  4362,     7,
            28,  3059,    19,  9094,  6214, 17745, 12203,     6, 26354, 42115,
             6,     8,  2388,  5910,     7,     5,  9161,  2591, 26914,    30,
          6247, 43814,    12,  1646, 16968,     4,     2]])
summarize: Omicron variant contains more than 32 amino acid mutations in the spike protein, including multiple vital amino acid mutation (K417N, T478K, E484A, N501Y, and D614G) that have been already detected in other VOCs of SAR

In [8]:
from transformers import pipeline

2024-07-15 14:11:16.807462: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-15 14:11:16.843134: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-15 14:11:16.843170: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-15 14:11:16.843211: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-15 14:11:16.851255: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: A

In [9]:
# Load the BART summarization pipeline
summarizer = pipeline("summarization", model='GanjinZero/biobart-large', tokenizer='GanjinZero/biobart-large')

/home/david.yang1/.local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [11]:
summary1 = summarizer(text, max_length=150, min_length=50, length_penalty=2.0, num_beams=4, truncation=True)

In [12]:
summary1

[{'summary_text': 'Nirmatrelvir-resistant SARS-CoV-2 variants with high fitness in an infectious cell culture systemThe oral protease inhibitor nirmatrevir is of key importance for prevention of severe coronavirus disease 2019 (COVID-19). To facilitate resistance monitoring, we studied severe acute respiratory syndrome coronavir 2 (SARS-Cov-2) explain resistance–associated substitutions in the transporter1 and combinationiv1 demonstrated respectively, a fitness level of ECbtiv0.1, The variants to remivelvir and human kidney concentrations. AND median-2 AND human kidney differences. Reverse genetics revealed that E166V and L50F\xa0+\xa0E166V conferred high resistance in infectious'}]

In [ ]:
text

In [15]:
from transformers import BartForQuestionAnswering, pipeline
import torch

In [16]:
qnamodel = BartForQuestionAnswering.from_pretrained(model_name)

Some weights of BartForQuestionAnswering were not initialized from the model checkpoint at GanjinZero/biobart-large and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
question = "What are the effects of the mutation E166V?"
text = flagged_papers["text"].iloc[0]

In [19]:
## make Q/A pipeline (pretrained bert model from hugging face) to extract procedure from raw text
qa_pipeline = pipeline(
    "question-answering",
    model='GanjinZero/biobart-large',
    tokenizer='GanjinZero/biobart-large'
)

Some weights of BartForQuestionAnswering were not initialized from the model checkpoint at GanjinZero/biobart-large and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
def procedure(text, question):
    result = []
    result.append(qa_pipeline({
    'context': text,
    'question': question
}))
    return {'Question':question,'Answer':result[0]['answer'], 'Score':result[0]['score']}

In [21]:
result=procedure(text, question)
result

{'Question': 'What are the effects of the mutation E166V?',
 'Answer': 'for titratable',
 'Score': 0.00039796289638616145}

In [32]:
result2=procedure(text, "Was any mutations mentioned?")
result2

{'Question': 'Was any mutations mentioned?',
 'Answer': '156152/download.Effect of oral nirmatrelvir on long COVID',
 'Score': 0.0003184566448908299}

In [33]:
result3=procedure(text, "what virus was discussed in this paper?")
result3

{'Question': 'what virus was discussed in this paper?',
 'Answer': '156152/download.Effect of oral nirmatrelvir on long COVID',
 'Score': 0.0003479756705928594}

# Train BioBART model

In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd

In [3]:
data = pd.read_csv("../data/pipeline_data/pokay.csv")

In [6]:
X = data["text"]
y = data["summary"]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)